In [ ]:
import torch
import detectron2
from unilm.dit.object_detection.ditod import add_vit_config

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
from PIL import Image

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances

In [ ]:
register_coco_instances("inhouse_train", {}, "/home/ec2-user/data/ocr_annotated_inhouse_data_coco/train/coco.json", "/home/ec2-user/data/ocr_annotated_inhouse_data_coco/train")
register_coco_instances("inhouse_val", {}, "/home/ec2-user/data/ocr_annotated_inhouse_data_coco/val/coco.json", "/home/ec2-user/data/ocr_annotated_inhouse_data_coco/val")

In [ ]:
import os
from unilm.dit.object_detection.ditod import MyTrainer

# instantiate config
cfg = get_cfg()
add_vit_config(cfg)
cfg.merge_from_file("/home/ec2-user/dit-document-layout-analysis/unilm/dit/object_detection/publaynet_configs/maskrcnn/maskrcnn_dit_base.yaml")

# load dataset
cfg.DATASETS.TRAIN = ("inhouse_train",)
cfg.DATASETS.TEST = ("inhouse_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# add model weights URL to config
cfg.MODEL.WEIGHTS = "/home/ec2-user/model/publaynet_dit-b_mrcnn.pth"

# set device
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# train specs
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 252000   # 300 epoch equal to max_iteration(252000) * batch_size(2)) / images_count(1680)
cfg.SOLVER.STEPS = []        # do not decay learning rate
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # just blob

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output